In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
import torchvision
from torch.utils.data import DataLoader, random_split
import numpy as np
from tqdm import tqdm
import os

In [3]:
# Device configuration GPU/CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
import numpy as np
print(np.__version__)


1.26.4


## VGG-16 Model


In [5]:
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )



    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [6]:
# # Hyperparameters
num_classes = 10
# num_epochs = 30
batch_size = 64
# learning_rate = 0.01

## Dataset

In [7]:
# Load CIFAR-100 dataset
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2761))
])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Split dataset into train and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
for inputs, labels in train_loader:
    print(f"Input shape: {inputs.shape}, Labels: {labels.shape}")
    break

Input shape: torch.Size([64, 3, 32, 32]), Labels: torch.Size([64])


In [9]:
# model = VGG16(num_classes=num_classes).to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)

In [10]:
# # Hyperparameters
# num_epochs = 30
# batch_size = 64
# learning_rate = 0.01

num_classes = 10
config_1 = {
    "num_epochs": 50,
    "batch_size": 128,
    "learning_rate": 0.001,
    "optimizer": "Adam",
    "scheduler": {
        "type": "StepLR",
        "step_size": 15,
        "gamma": 0.1
    }
}

config_2 = {
    "num_epochs": 40,
    "batch_size": 64,
    "learning_rate": 0.01,
    "optimizer": "SGD",
    "momentum": 0.9,
    "scheduler": {
        "type": "CosineAnnealingLR",
        "T_max": 40
    }
}

config_3 = {
    "num_epochs": 100,
    "batch_size": 32,
    "learning_rate": 0.0005,
    "optimizer": "Adam",
    "scheduler": {
        "type": "ReduceLROnPlateau",
        "mode": "min",
        "factor": 0.1,
        "patience": 10
    },
    "regularization": {
        "dropout_rate": 0.5
    }
}

def config_test(selected_config):
    # Atribuindo os hiperparâmetros escolhidos
    num_epochs = selected_config["num_epochs"]
    batch_size = selected_config["batch_size"]
    learning_rate = selected_config["learning_rate"]

    model = VGG16(num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()

    if selected_config["optimizer"] == "Adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    elif selected_config["optimizer"] == "SGD":
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=selected_config.get("momentum", 0))

    # Configurando o scheduler baseado na configuração
    if selected_config["scheduler"]["type"] == "StepLR":
        scheduler = torch.optim.lr_scheduler.StepLR(
            optimizer,
            step_size=selected_config["scheduler"]["step_size"],
            gamma=selected_config["scheduler"]["gamma"]
        )
    elif selected_config["scheduler"]["type"] == "CosineAnnealingLR":
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=selected_config["scheduler"]["T_max"]
        )
    elif selected_config["scheduler"]["type"] == "ReduceLROnPlateau":
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode=selected_config["scheduler"]["mode"],
            factor=selected_config["scheduler"]["factor"],
            patience=selected_config["scheduler"]["patience"]
        )


    return num_epochs, batch_size, learning_rate, model, criterion, optimizer, scheduler


# Training Original VGG16

In [11]:
def validate(model, dataloader):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(dataloader)
    val_accuracy = 100 * correct / total
    return val_loss, val_accuracy

In [12]:
def train(train_loader, val_loader, num_epochs, model, criterion, optimizer, scheduler, learning_rate=1.5, batch_size=10):
  total_step = len(train_loader)
  model.to(device)
  model.train()
  for epoch in range(num_epochs):

      running_loss = 0.0
      for i, (inputs, labels) in enumerate(train_loader):
          inputs, labels = inputs.to(device), labels.to(device)

          outputs = model(inputs)
          loss = criterion(outputs, labels)
          optimizer.zero_grad()
          loss.backward()

          optimizer.step()

          running_loss += loss.item()
          if i % 100 == 199:  # Print every 100 mini-batches
              print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
              running_loss = 0.0

      # Validate the model
      val_loss , val_accuracy = validate(model, val_loader)
      print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

      scheduler.step()

  return model

In [13]:
def calculate_accuracy(model, dataloader):
  model.to(device)
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
      for inputs, labels in dataloader:
          inputs, labels = inputs.to(device), labels.to(device)
          outputs = model(inputs)
          _, predicted = torch.max(outputs, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  accuracy = 100 * correct / total
  return accuracy

### Test 1

In [ ]:
num_epochs, batch_size, learning_rate, model, criterion, optimizer, scheduler = config_test(config_1)
model = train(train_loader, val_loader, num_epochs, model, criterion, optimizer, scheduler)
torch.save(model.state_dict(), "model_t2_test.h5")

The code in the cell looks correct and should work without any errors, given that all the necessary variables and modules are already defined and imported in the previous cells. However, to ensure that the code runs smoothly, you should make sure that the `train_loader`, `val_loader`, `model`, `criterion`, `optimizer`, `device`, and `num_epochs` are properly defined and imported.



Made changes.

In [ ]:
accuracy = calculate_accuracy(model, test_loader)
print(f'Test Accuracy: {accuracy:.2f}%')

### Test 2

In [ ]:
num_epochs, batch_size, learning_rate, model, criterion, optimizer, scheduler = config_test(config_2)
model = train(train_loader, val_loader, num_epochs, model, criterion, optimizer, scheduler)
torch.save(model.state_dict(), "model_t2_test2.h5")

In [ ]:
accuracy = calculate_accuracy(model, test_loader)
print(f'Test Accuracy: {accuracy:.2f}%')

### Test 3

In [ ]:
num_epochs, batch_size, learning_rate, model, criterion, optimizer, scheduler = config_test(config_3)
model = train(train_loader, val_loader, num_epochs, model, criterion, optimizer, scheduler)
torch.save(model.state_dict(), "model_t2_test3.h5")

In [ ]:
accuracy = calculate_accuracy(model, test_loader)
print(f'Test Accuracy: {accuracy:.2f}%')

# Training Pruned VGG16

### Prune models

In [14]:
import torch.nn.utils.prune as prune
import torch.nn.functional as F


# Function to apply pruning to specific layers -> LOCAL
def apply_pruning(model, conv_layers_, pruning_method, amount, structured=False):
  model.to(device)
  model.train()
  for name, layer in conv_layers_:
    if isinstance(layer, nn.Conv2d):
        if structured:
            if pruning_method == 'l1_structured':
                prune.ln_structured(layer, name='weight', amount=amount, n=1, dim=0)  # Prune output channels
                print(f"Applied L1 Structured pruning to layer: {name}")
            elif pruning_method == 'l2_structured':
                prune.ln_structured(layer, name='weight', amount=amount, n=2, dim=0)
                print(f"Applied L2 Structured pruning to layer: {name}")
            elif pruning_method == 'random_structured':
                prune.random_structured(layer, name='weight', amount=amount, dim=0)
                print(f"Applied Random Structured pruning to layer: {name}")
            else:
                raise ValueError(f"Unsupported structured pruning method: {pruning_method}")
        else:
            if pruning_method == 'l1_unstructured':
                prune.l1_unstructured(layer, name='weight', amount=amount)
                print(f"Applied L1 Unstructured pruning to layer: {name}")
            # elif pruning_method == 'l2_unstructured':
            #     prune.l2_unstructured(layer, name='weight', amount=amount)
            #     print(f"Applied L2 Unstructured pruning to layer: {name}")
            elif pruning_method == 'random_unstructured':
                prune.random_unstructured(layer, name='weight', amount=amount)
                print(f"Applied Random Unstructured pruning to layer: {name}")
            else:
                raise ValueError(f"Unsupported unstructured pruning method: {pruning_method}")

  return model

In [15]:
# Function to enforce pruning masks
def enforce_pruning_masks(model):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            if hasattr(module, 'weight_mask'):
                module.weight.data.mul_(module.weight_mask)


In [16]:
# Get all convolutional layers in VGG16
def get_conv_layers(model):
    conv_layers = []
    for name, module in model.features.named_modules():
        if isinstance(module, nn.Conv2d):
            conv_layers.append((name, module))
    return conv_layers


In [17]:
def load_model(model_path, num_classes=10):
    model = VGG16(num_classes)
    model.load_state_dict(torch.load(model_path, map_location=device))
    return model


In [18]:
# def load_model_with_masks(model, file_path):
#     # Load the state_dict with masks
#     state_with_masks = torch.load(file_path, map_location=device)
#     model.load_state_dict(state_with_masks, strict=False)  # Load weights without strict check

#     model.to(device)

#     # Reapply masks to the layers
#     for name, module in model.named_modules():
#         if isinstance(module, nn.Conv2d):
#             mask_key = f"{name}.weight_mask"
#             if mask_key in state_with_masks:
#                 mask = state_with_masks[mask_key].to(device)
#                 prune.custom_from_mask(module, name='weight', mask=mask)

#     return model

def load_model_with_masks(model, file_path):
    # Carrega o estado salvo, incluindo máscaras
    state_with_masks = torch.load(file_path, map_location=device)    
    # Carrega os pesos sem forçar correspondência completa
    model.load_state_dict(state_with_masks, strict=False)
    model.to(device)

    # Reaplica as máscaras às camadas Conv2d
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            mask_key = f"{name}.weight_mask"
            if mask_key in state_with_masks:
                mask = state_with_masks[mask_key].to(device)
                prune.custom_from_mask(module, name='weight', mask=mask)
                print(f"Máscara reaplicada para {name}.")
            else:
                print(f"Máscara não encontrada para {name}. Verifique o arquivo salvo.")

    return model


def check_pruning_applied(model):
    pruned_layers = []
    unpruned_layers = []

    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):  # Verificar apenas camadas Conv2d
            if hasattr(module, 'weight_orig'):  # Verifica se o pruning foi aplicado
                pruned_layers.append(name)
            else:
                unpruned_layers.append(name)
    
    return pruned_layers, unpruned_layers
    

def save_model_with_masks(model, file_path):
    # Include pruning masks in the state_dict
    state_with_masks = model.state_dict()
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            if hasattr(module, 'weight_mask'):
                state_with_masks[f"{name}.weight_mask"] = module.weight_mask

    torch.save(state_with_masks, file_path)


In [19]:
models_path = "models/"

#### Pruning Conv2d layers - 10%

In [ ]:
pruning_methods_all = ['l1_structured', 'l2_structured', 'random_structured', 'l1_unstructured', 'random_unstructured']
pruning_methods_structured = ['l1_structured', 'l2_structured', 'random_structured']
pruning_methods_unstructured = ['l1_unstructured', 'random_unstructured']

for method in pruning_methods_all:
  # Load pretrained model
  model = load_model("/content/model_t2.h5")

  # Prune 20% of weights in all Conv2d layers
  conv_layers = get_conv_layers(model)
  if method in pruning_methods_structured:
    apply_pruning(model, conv_layers, pruning_method=f"{method}", amount=0.1, structured=True)
  elif method in pruning_methods_unstructured:
    apply_pruning(model, conv_layers, pruning_method=f"{method}", amount=0.1, structured=False)

  # # Consolidate pruning
  # for layer in model.modules():
  #     if isinstance(layer, (nn.Conv2d)):
  #         prune.remove(layer, 'weight')

  save_model_with_masks(model, f'{models_path}vgg16_pruned_allConv2dlayers_{method}_0.1_with_masks.pth')


#### Pruning Conv2d layers - 20%

In [ ]:
pruning_methods_all = ['l1_structured', 'l2_structured', 'random_structured', 'l1_unstructured', 'random_unstructured']
pruning_methods_structured = ['l1_structured', 'l2_structured', 'random_structured']
pruning_methods_unstructured = ['l1_unstructured', 'random_unstructured']

for method in pruning_methods_all:
    # Load pretrained model
    model = load_model("/content/model_t2.h5")
    
    # Prune 20% of weights in all Conv2d layers
    conv_layers = get_conv_layers(model)
        if method in pruning_methods_structured:
    apply_pruning(model, conv_layers, pruning_method=f"{method}", amount=0.2, structured=True)
        elif method in pruning_methods_unstructured:
    apply_pruning(model, conv_layers, pruning_method=f"{method}", amount=0.2, structured=False)
    
    # # Consolidate pruning
    # for layer in model.modules():
    #     if isinstance(layer, (nn.Conv2d)):
    #         prune.remove(layer, 'weight')
    
    torch.save(model.state_dict(), f'vgg16_pruned_allConv2dlayers_{method}_0.2.pth')
    save_model_with_masks(model, f'vgg16_pruned_allConv2dlayers_{method}_0.1_with_masks.pth')


#### Pruning Conv2d layers - 30%

In [ ]:
pruning_methods_all = ['l1_structured', 'l2_structured', 'random_structured', 'l1_unstructured', 'random_unstructured']
pruning_methods_structured = ['l1_structured', 'l2_structured', 'random_structured']
pruning_methods_unstructured = ['l1_unstructured', 'random_unstructured']

for method in pruning_methods_all:
  # Load pretrained model
  model = load_model("/content/model_t2.h5")

  # Prune 20% of weights in all Conv2d layers
  conv_layers = get_conv_layers(model)
  if method in pruning_methods_structured:
    apply_pruning(model, conv_layers, pruning_method=f"{method}", amount=0.3, structured=True)
  elif method in pruning_methods_unstructured:
    apply_pruning(model, conv_layers, pruning_method=f"{method}", amount=0.3, structured=False)

  # # Consolidate pruning
  # for layer in model.modules():
  #     if isinstance(layer, (nn.Conv2d)):
  #         prune.remove(layer, 'weight')

  torch.save(model.state_dict(), f'vgg16_pruned_allConv2dlayers_{method}_0.3.pth')

### Fine Tune models for comparison

In [33]:
import csv
from tqdm import tqdm

# Função de treinamento ajustada
def finetune(train_loader, val_loader, num_epochs, model, criterion, optimizer, scheduler, pruned_model_name, learning_rate=1.5, batch_size=10):
    total_step = len(train_loader)
    model.to(device)
    model.train()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)


    # Lista para armazenar os logs
    logs = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()

            running_loss += loss.item()
            
            if i % 100 == 199:  # Salva a média a cada 100 mini-batches
                avg_loss = running_loss / 100
                logs.append({
                    "epoch": epoch + 1,
                    "step": i + 1,
                    "train_loss": avg_loss
                })
                running_loss = 0.0

        # Validação do modelo
        val_loss, val_accuracy = validate(model, val_loader)
        logs.append({
            "epoch": epoch + 1,
            "validation_loss": val_loss,
            "validation_accuracy": val_accuracy
        })

        scheduler.step()

    # Salva os logs em um arquivo CSV
    with open(f"logs/models_finetuned_frozen/logs_{pruned_model_name}.csv", "w", newline="") as csvfile:
        fieldnames = ["epoch", "validation_loss", "validation_accuracy"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for log in logs:
            print(log)
            writer.writerow(log)

    # Verifica o estado do pruning após o fine-tuning
    pruned_layers, unpruned_layers = check_pruning_applied(model)
    print(f"Após o treinamento: Camadas podadas ({len(pruned_layers)}): {pruned_layers}")
    print(f"Camadas não podadas ({len(unpruned_layers)}): {unpruned_layers}")

    return model


In [30]:
models_folder = "models_pruned_with_mask/"
models_buffer = {}

pruning_methods_all = ['l1_structured', 'l2_structured', 'random_structured', 'l1_unstructured', 'random_unstructured']

for method in pruning_methods_all:
  for percentage in [0.1, 0.2, 0.3]:
    model = load_model("model_t2.h5")
    model.to(device)
    models_buffer[f"vgg16_pruned_allConv2dlayers_{method}_{percentage}"] = load_model_with_masks(model, f"{models_folder}vgg16_pruned_allConv2dlayers_{method}_{percentage}.pth")
    print()
      

Máscara reaplicada para features.0.
Máscara reaplicada para features.3.
Máscara reaplicada para features.7.
Máscara reaplicada para features.10.
Máscara reaplicada para features.14.
Máscara reaplicada para features.17.
Máscara reaplicada para features.20.
Máscara reaplicada para features.24.
Máscara reaplicada para features.27.
Máscara reaplicada para features.30.
Máscara reaplicada para features.34.
Máscara reaplicada para features.37.
Máscara reaplicada para features.40.

Máscara reaplicada para features.0.
Máscara reaplicada para features.3.
Máscara reaplicada para features.7.
Máscara reaplicada para features.10.
Máscara reaplicada para features.14.
Máscara reaplicada para features.17.
Máscara reaplicada para features.20.
Máscara reaplicada para features.24.
Máscara reaplicada para features.27.
Máscara reaplicada para features.30.
Máscara reaplicada para features.34.
Máscara reaplicada para features.37.
Máscara reaplicada para features.40.

Máscara reaplicada para features.0.
Máscar

In [32]:
import torch.nn.utils.prune as prune
torch.cuda.empty_cache()

# Função para verificar se o pruning foi aplicado
def check_pruning_applied(model):
    pruned_layers = []
    unpruned_layers = []
    
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d):  # Verificar apenas camadas Conv2d
            if hasattr(module, 'weight_mask'):
                pruned_layers.append(name)
            else:
                unpruned_layers.append(name)
    
    return pruned_layers, unpruned_layers

# Iterar sobre todos os modelos no buffer e verificar o pruning
for pruned_model_name in models_buffer.keys():
    print(f"\nVerificando o modelo: {pruned_model_name}")
    model = models_buffer[pruned_model_name]
    
    pruned_layers, unpruned_layers = check_pruning_applied(model)
    
    print(f"Camadas podadas ({len(pruned_layers)}): {pruned_layers}")
    print(f"Camadas não podadas ({len(unpruned_layers)}): {unpruned_layers}")

            


Verificando o modelo: vgg16_pruned_allConv2dlayers_l1_structured_0.1
Camadas podadas (13): ['features.0', 'features.3', 'features.7', 'features.10', 'features.14', 'features.17', 'features.20', 'features.24', 'features.27', 'features.30', 'features.34', 'features.37', 'features.40']
Camadas não podadas (0): []

Verificando o modelo: vgg16_pruned_allConv2dlayers_l1_structured_0.2
Camadas podadas (13): ['features.0', 'features.3', 'features.7', 'features.10', 'features.14', 'features.17', 'features.20', 'features.24', 'features.27', 'features.30', 'features.34', 'features.37', 'features.40']
Camadas não podadas (0): []

Verificando o modelo: vgg16_pruned_allConv2dlayers_l1_structured_0.3
Camadas podadas (13): ['features.0', 'features.3', 'features.7', 'features.10', 'features.14', 'features.17', 'features.20', 'features.24', 'features.27', 'features.30', 'features.34', 'features.37', 'features.40']
Camadas não podadas (0): []

Verificando o modelo: vgg16_pruned_allConv2dlayers_l2_struct

In [22]:
# file_path = 'models_pruned_with_mask/vgg16_pruned_allConv2dlayers_l1_structured_0.1.pth'

# model = load_model("model_t2.h5")

# model.to(device)

# model = load_model_with_masks(model, file_path)

In [ ]:
#using config 2 since it is the config of best professor model
num_epochs, batch_size, learning_rate, model, criterion, optimizer, scheduler = config_test(config_2)

for pruned_model_name in models_buffer.keys():
    print(f"\nTreinando o modelo: {pruned_model_name}")
    model = models_buffer[pruned_model_name]

    # Treinamento do modelo
    pruned_finetuned_model = finetune(train_loader, val_loader, num_epochs, model, criterion, optimizer, scheduler, pruned_model_name)

    # Verifica e remove pruning (consolidação)
    for layer in pruned_finetuned_model.modules():
        if isinstance(layer, nn.Conv2d) and hasattr(layer, 'weight_orig'):
            prune.remove(layer, 'weight')
            print(f"Pruning removido da camada {layer}.")
        else:
            print(f"Camada {layer} não possui pruning para remover.")

    # Salva o modelo ajustado
    torch.save(pruned_finetuned_model.state_dict(), f"models_finetuned_frozen_alt/{pruned_model_name}_finetuned_frozen.pth")


Treinando o modelo: vgg16_pruned_allConv2dlayers_l1_structured_0.1


Epoch 16/40: 100%|██████████| 625/625 [00:20<00:00, 30.69it/s]


In [ ]:
models_buffer = {}
for method in pruning_methods_all:
  for percentage in [0.1, 0.2, 0.3]:
    model = load_model("model_t2.h5")
    model.to(device)
    models_buffer[f"{models_folder}vgg16_pruned_allConv2dlayers_{method}_{percentage}"] = load_model_with_masks(model, f"{models_folder}vgg16_pruned_allConv2dlayers_{method}_{percentage}.pth")

models_buffer


for pruned_model_name in models_buffer.keys():
  for layer in models_buffer[pruned_model_name].modules():
      if isinstance(layer, (nn.Conv2d)):
          prune.remove(layer, 'weight')
  torch.save(models_buffer[pruned_model_name].state_dict(), f"{pruned_model_name}_frozen.pth")


In [ ]:
def train_student(student_model, teacher_model, train_loader, val_loader, num_epochs, soft_target_loss_weight, ce_loss_weight, temperature, learning_rate, scheduler):
    student_model.train()
    teacher_model.eval()
    optimizer = torch.optim.SGD(student_model.parameters(), lr=learning_rate, momentum=0.9)

    ce_loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            # Forward pass through teacher model
            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)
                teacher_probs = nn.functional.softmax(teacher_outputs / temperature, dim=1)

            # Forward pass through student model
            student_outputs = student_model(inputs)
            student_probs = nn.functional.log_softmax(student_outputs / temperature, dim=1)

            # Compute distillation loss
            soft_target_loss = torch.sum(teacher_probs * (teacher_probs.log() - student_probs))/ student_probs.size()[0] * (temperature**2)

            label_loss = ce_loss(student_outputs, labels)

            loss = soft_target_loss_weight * soft_target_loss + ce_loss_weight * label_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:  # Print every 100 mini-batches
                print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
                running_loss = 0.0

        # Validate the student model
        student_model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student_model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

        student_model.train()
        scheduler.step()

In [ ]:
num_epochs, batch_size, learning_rate, model, criterion, optimizer, scheduler = config_test(config_2)

models_buffer = {}
teacher = load_model("model_t2.h5")
teacher.eval()


for method in pruning_methods_all:
  for percentage in [0.1, 0.2, 0.3]:
    models_buffer[f"{models_folder}vgg16_pruned_allConv2dlayers_{method}_{percentage}"] = load_model_with_masks(model, f"{models_folder}vgg16_pruned_allConv2dlayers_{method}_{percentage}.pth")


for pruned_model_name in models_buffer.keys():
    print(f"\nTreinando o modelo com KD: {pruned_model_name}")
    student = models_buffer[pruned_model_name]
    student.train()
    # train_student(student, teacher, train_loader, val_loader, num_epochs,0.25, 0.75, 2, optimizer)
    train_student(student, teacher, train_loader, val_loader, num_epochs,0.25, 0.75, 2, learning_rate, scheduler)

    torch.save(student, f"{models_folder}/{pruned_model_name}_with_KD.pth")

In [ ]:
# Model size
def get_model_size(model_path):
    size = os.path.getsize(model_path) / (1024 * 1024)  # Convert to MB
    return size

# Sparsity
def calculate_sparsity(model):
    total_weights = 0
    zero_weights = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            total_weights += param.numel()
            zero_weights += torch.sum(param == 0).item()
    sparsity = 100.0 * zero_weights / total_weights
    return sparsity


In [ ]:
import time
import torch

def get_inference_time(model, test_loader, device, num_iterations=100):
    model.eval()
    total_time = 0.0
    with torch.no_grad():
        # Warm-up
        for _ in range(10):
            try:
                inputs, _ = next(iter(test_loader))
            except StopIteration:
                break
            inputs = inputs.to(device)
            outputs = model(inputs)

        # Timing
        for i, (inputs, _) in enumerate(test_loader):
            if i >= num_iterations:
                break
            inputs = inputs.to(device)
            start_time = time.time()
            outputs = model(inputs)
            if device.type == 'cuda':
                torch.cuda.synchronize()  # Wait for GPU operations to finish
            end_time = time.time()
            total_time += (end_time - start_time)

    avg_time_per_batch = total_time / num_iterations
    return avg_time_per_batch
    

In [ ]:
def get_evaluation(pruning_methods_all, percentages, test_loader, device, inference_iterations = 100):

  models_buffer = {
        'pruned': {},
        'finetuned': {},
        'kd':{}
    }
  

  for method in pruning_methods_all:

    models_buffer['pruned'][method] = []
    models_buffer['finetuned'][method] = []
    models_buffer['kd'][method] = []

    for percentage in percentages:

      # FROZEN - PRUNED ONLY
      model_name = f"vgg16_pruned_allConv2dlayers_{method}_{percentage}_frozen"
      model_path = f"{models_folder}/{model_name}.pth"

      if os.path.exists(model_path):

        # Load model - frozen (only pruned)
        model = load_model(model_path)

        # Evaluate - frozen (only pruned)
        accuracy = calculate_accuracy(model, test_loader)
        model_size = get_model_size(model_path)
        sparsity = calculate_sparsity(model)
        inference_time = get_inference_time(model, test_loader, device, inference_iterations)

        # Save evaluation - frozen (only pruned)
        models_buffer['pruned'][method].append({
                'name' : model_name,
                'pruning_percentage' : percentage,
                'accuracy' : accuracy,
                'model_size_MB' : model_size,
                'sparsity' : sparsity,
                'inference_time' : inference_time
              })

        print(f"Evaluated Pruned Model: {model_name}")
        print(f"Accuracy: {accuracy:.2f}%, Size: {model_size:.2f} MB, Sparsity: {sparsity:.2f}%, Inference Time: {inference_time:.6f} sec/batch\n")
      else:
        print(f"Pruned model file not found: {model_path}")
        continue


      # PRUNED AND FINETUNED
      model_name = f"vgg16_pruned_allConv2dlayers_{method}_{percentage}_finetuned_frozen"
      model_path = f"{models_folder}/{model_name}.pth"

      if os.path.exists(model_path):

        # Load model - finetuned
        model = load_model(model_path)

        # Evaluate - finetuned
        accuracy = calculate_accuracy(model, test_loader)
        model_size = get_model_size(model_path)
        sparsity = calculate_sparsity(model)
        inference_time = get_inference_time(model, test_loader, device, inference_iterations)


        # Save evaluation - finetuned
        models_buffer['finetuned'][method].append({
                'name' : model_name,
                'pruning_percentage' : percentage,
                'accuracy' : accuracy,
                'model_size_MB' : model_size,
                'sparsity' : sparsity,
                'inference_time' : inference_time
              })

        print(f"Evaluated Pruned Model: {model_name}")
        print(f"Accuracy: {accuracy:.2f}%, Size: {model_size:.2f} MB, Sparsity: {sparsity:.2f}%, Inference Time: {inference_time:.6f} sec/batch\n")
      else:
        print(f"Pruned model file not found: {model_path}")
        continue


      #  PRUNED AND KD
      model_name = f"vgg16_pruned_allConv2dlayers_{method}_{percentage}_with_KD.pth"
      model_path = f"{models_folder}/{model_name}.pth"

      if os.path.exists(model_path):

    #    Load model - kd
        model = load_model(model_path)

    #    Evaluate - kd
        accuracy = calculate_accuracy(model, test_loader)
        model_size = get_model_size(model_path)
        sparsity = calculate_sparsity(model)
        inference_time = get_inference_time(model, test_loader, device, inference_iterations)


    #    Save evaluation - kd
        models_buffer['kd'][method].append({
                'name' : model_name,
                'pruning_percentage' : percentage,
                'accuracy' : accuracy,
                'model_size_MB' : model_size,
                'sparsity' : sparsity,
                'inference_time' : inference_time
              })

        print(f"Evaluated Pruned Model: {model_name}")
        print(f"Accuracy: {accuracy:.2f}%, Size: {model_size:.2f} MB, Sparsity: {sparsity:.2f}%, Inference Time: {inference_time:.6f} sec/batch\n")
      else:
        print(f"Pruned model file not found: {model_path}")
        continue

  return models_buffer


In [ ]:
evaluation = get_evaluation(pruning_methods_all, [0.1, 0.2, 0.3], test_loader, device, 100)


In [ ]:
# Save evaluation in pickle file
import pickle

# Create the directory if it doesn't exist
os.makedirs('evaluations', exist_ok=True)

# Define the file path with a meaningful name
pickle_file_path = 'evaluations/evaluation_pruning_results.pkl'

# Save evaluation dictionary
with open(pickle_file_path, 'wb') as file:
    pickle.dump(evaluation, file)

print(f"Evaluation results saved successfully at '{pickle_file_path}'.")

Comparison

comparing inference time, memory consumption, and model performance.

# Evaluate pruned, pruned and finetuned and pruned and KD models
(TODO add KD on evaluation)

In [ ]:
# Model size
import os

def get_model_size(model_path):
    size = os.path.getsize(model_path) / (1024 * 1024)  # Convert to MB
    return size

In [ ]:
# Sparsity
def calculate_sparsity(model):
    total_weights = 0
    zero_weights = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            total_weights += param.numel()
            zero_weights += torch.sum(param == 0).item()
    sparsity = 100.0 * zero_weights / total_weights
    return sparsity

In [ ]:
import time
import torch

def get_inference_time(model, test_loader, device, num_iterations=100):
    model.eval()
    total_time = 0.0
    with torch.no_grad():
        # Warm-up
        for _ in range(10):
            try:
                inputs, _ = next(iter(test_loader))
            except StopIteration:
                break
            inputs = inputs.to(device)
            outputs = model(inputs)

        # Timing
        for i, (inputs, _) in enumerate(test_loader):
            if i >= num_iterations:
                break
            inputs = inputs.to(device)
            start_time = time.time()
            outputs = model(inputs)
            if device.type == 'cuda':
                torch.cuda.synchronize()  # Wait for GPU operations to finish
            end_time = time.time()
            total_time += (end_time - start_time)

    avg_time_per_batch = total_time / num_iterations
    return avg_time_per_batch


In [ ]:
def get_evaluation(pruning_methods_all, percentages, test_loader, device, inference_iterations = 100):

  models_buffer = {
        'pruned': {},
        'finetuned': {},
        'kd': {}
    }
  

  for method in pruning_methods_all:

    models_buffer['pruned'][method] = []
    models_buffer['finetuned'][method] = []
    models_buffer['kd'][method] = []

    for percentage in percentages:

      # FROZEN - PRUNED ONLY
      model_name = f"vgg16_pruned_allConv2dlayers_{method}_{percentage}_frozen"
      model_path = f"/content/{model_name}.pth"

      if os.path.exists(model_path):

        # Load model - frozen (only pruned)
        model = load_model(model_path)

        # Evaluate - frozen (only pruned)
        accuracy = calculate_accuracy(model, test_loader)
        model_size = get_model_size(model_path)
        sparsity = calculate_sparsity(model)
        inference_time = get_inference_time(model, test_loader, device, inference_iterations)

        # Save evaluation - frozen (only pruned)
        models_buffer['pruned'][method].append({
                'name' : model_name,
                'pruning_percentage' : percentage,
                'accuracy' : accuracy,
                'model_size_MB' : model_size,
                'sparsity' : sparsity,
                'inference_time' : inference_time
              })

        print(f"Evaluated Pruned Model: {model_name}")
        print(f"Accuracy: {accuracy:.2f}%, Size: {model_size:.2f} MB, Sparsity: {sparsity:.2f}%, Inference Time: {inference_time:.6f} sec/batch\n")
      else:
        print(f"Pruned model file not found: {model_path}")
        continue


      # PRUNED AND FINETUNED
      model_name = f"vgg16_pruned_allConv2dlayers_{method}_{percentage}_finetuned_frozen"
      model_path = f"/content/{model_name}.pth"

      if os.path.exists(model_path):

        # Load model - finetuned
        model = load_model(model_path)

        # Evaluate - finetuned
        accuracy = calculate_accuracy(model, test_loader)
        model_size = get_model_size(model_path)
        sparsity = calculate_sparsity(model)
        inference_time = get_inference_time(model, test_loader, device, inference_iterations)


        # Save evaluation - finetuned
        models_buffer['finetuned'][method].append({
                'name' : model_name,
                'pruning_percentage' : percentage,
                'accuracy' : accuracy,
                'model_size_MB' : model_size,
                'sparsity' : sparsity,
                'inference_time' : inference_time
              })

        print(f"Evaluated Pruned Model: {model_name}")
        print(f"Accuracy: {accuracy:.2f}%, Size: {model_size:.2f} MB, Sparsity: {sparsity:.2f}%, Inference Time: {inference_time:.6f} sec/batch\n")
      else:
        print(f"Pruned model file not found: {model_path}")
        continue


      # PRUNED AND KD
      model_name = f"vgg16_pruned_allConv2dlayers_{method}_{percentage}_with_KD.pth"
      model_path = f"/content/{model_name}.pth"

      if os.path.exists(model_path):

        # Load model - kd
        model = load_model(model_path)

        # Evaluate - kd
        accuracy = calculate_accuracy(model, test_loader)
        model_size = get_model_size(model_path)
        sparsity = calculate_sparsity(model)
        inference_time = get_inference_time(model, test_loader, device, inference_iterations)


        # Save evaluation - kd
        models_buffer['kd'][method].append({
                'name' : model_name,
                'pruning_percentage' : percentage,
                'accuracy' : accuracy,
                'model_size_MB' : model_size,
                'sparsity' : sparsity,
                'inference_time' : inference_time
              })

        print(f"Evaluated Pruned Model: {model_name}")
        print(f"Accuracy: {accuracy:.2f}%, Size: {model_size:.2f} MB, Sparsity: {sparsity:.2f}%, Inference Time: {inference_time:.6f} sec/batch\n")
      else:
        print(f"Pruned model file not found: {model_path}")
        continue

  return models_buffer

In [ ]:
evaluation = get_evaluation(pruning_methods_all, [0.1, 0.2, 0.3], test_loader, device, 100)

# Save evaluation in pickle file
import pickle
import os

# Create the directory if it doesn't exist
os.makedirs('evaluations', exist_ok=True)

# Define the file path with a meaningful name
pickle_file_path = 'evaluations/evaluation_pruning_results.pkl'

# Save evaluation dictionary
with open(pickle_file_path, 'wb') as file:
    pickle.dump(evaluation, file)

print(f"Evaluation results saved successfully at '{pickle_file_path}'.")

### Visualize Evaluation - ALL BY CHATGPT

In [ ]:
!pip install pandas matplotlib seaborn

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def buffer_to_dataframe(evaluation, category):
    records = []
    for method, models in evaluation.get(category, {}).items():
        for model_info in models:
            record = model_info.copy()
            record['pruning_method'] = method
            records.append(record)
    df = pd.DataFrame(records)
    return df

###########################
# # ONLY IF NEEDED -> load pickle file with evaluation
# import pickle

# # Define the pickle file path
# pickle_file_path = 'evaluations/evaluation_pruning_results.pkl'

# # Load the evaluation dictionary from the pickle file
# with open(pickle_file_path, 'rb') as file:
#     loaded_evaluation = pickle.load(file)

# print("Evaluation results loaded successfully.")
###########################

# Create DataFrames for each category
pruned_df = buffer_to_dataframe(evaluation, 'pruned')
finetuned_df = buffer_to_dataframe(evaluation, 'finetuned')
kd_df = buffer_to_dataframe(evaluation, 'kd')


In [ ]:
sns.set(style="whitegrid")
plt.rcParams.update({'font.size': 12})

In [ ]:
def plot_accuracy(df, category):
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df, x='pruning_percentage', y='accuracy', hue='pruning_method', marker='o')
    plt.title(f'Accuracy vs. Pruning Percentage ({category.capitalize()} Models)')
    plt.xlabel('Pruning Percentage')
    plt.ylabel('Accuracy (%)')
    plt.legend(title='Pruning Method')
    plt.show()

# Plot Accuracy for Each Category
plot_accuracy(pruned_df, 'pruned')
plot_accuracy(finetuned_df, 'finetuned')
plot_accuracy(kd_df, 'kd')


In [ ]:
def plot_inference_time(df, category):
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df, x='pruning_percentage', y='inference_time_sec', hue='pruning_method', marker='o')
    plt.title(f'Inference Time vs. Pruning Percentage ({category.capitalize()} Models)')
    plt.xlabel('Pruning Percentage')
    plt.ylabel('Inference Time per Batch (sec)')
    plt.legend(title='Pruning Method')
    plt.show()

# Plot Inference Time for Each Category
plot_inference_time(pruned_df, 'pruned')
plot_inference_time(finetuned_df, 'finetuned')
plot_inference_time(kd_df, 'kd')


In [ ]:
def plot_sparsity(df, category):
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df, x='pruning_percentage', y='sparsity_percent', hue='pruning_method', marker='o')
    plt.title(f'Sparsity vs. Pruning Percentage ({category.capitalize()} Models)')
    plt.xlabel('Pruning Percentage')
    plt.ylabel('Sparsity (%)')
    plt.legend(title='Pruning Method')
    plt.show()

# Plot Sparsity for Each Category
plot_sparsity(pruned_df, 'pruned')
plot_sparsity(finetuned_df, 'finetuned')
plot_sparsity(kd_df, 'kd')


In [ ]:
def plot_model_size(df, category):
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df, x='pruning_percentage', y='model_size_MB', hue='pruning_method', marker='o')
    plt.title(f'Model Size vs. Pruning Percentage ({category.capitalize()} Models)')
    plt.xlabel('Pruning Percentage')
    plt.ylabel('Model Size (MB)')
    plt.legend(title='Pruning Method')
    plt.show()

# Plot Model Size for Each Category
plot_model_size(pruned_df, 'pruned')
plot_model_size(finetuned_df, 'finetuned')
plot_model_size(kd_df, 'kd')


In [ ]:
# Add a 'category' column to each DataFrame
pruned_df['category'] = 'Pruned'
finetuned_df['category'] = 'Finetuned'
kd_df['category'] = 'KD'

# Concatenate all DataFrames
combined_df = pd.concat([pruned_df, finetuned_df, kd_df], ignore_index=True)


In [ ]:
plt.figure(figsize=(12, 8))
sns.lineplot(data=combined_df, x='pruning_percentage', y='accuracy', hue='category', style='category', markers=True, dashes=False)
plt.title('Accuracy Comparison Across Model Categories')
plt.xlabel('Pruning Percentage')
plt.ylabel('Accuracy (%)')
plt.legend(title='Model Category')
plt.show()


In [ ]:
plt.figure(figsize=(12, 8))
sns.lineplot(data=combined_df, x='pruning_percentage', y='inference_time_sec', hue='category', style='category', markers=True, dashes=False)
plt.title('Inference Time Comparison Across Model Categories')
plt.xlabel('Pruning Percentage')
plt.ylabel('Inference Time per Batch (sec)')
plt.legend(title='Model Category')
plt.show()


In [ ]:
plt.figure(figsize=(12, 8))
sns.lineplot(data=combined_df, x='pruning_percentage', y='sparsity_percent', hue='category', style='category', markers=True, dashes=False)
plt.title('Sparsity Comparison Across Model Categories')
plt.xlabel('Pruning Percentage')
plt.ylabel('Sparsity (%)')
plt.legend(title='Model Category')
plt.show()


In [ ]:
plt.figure(figsize=(12, 8))
sns.lineplot(data=combined_df, x='pruning_percentage', y='model_size_MB', hue='category', style='category', markers=True, dashes=False)
plt.title('Model Size Comparison Across Model Categories')
plt.xlabel('Pruning Percentage')
plt.ylabel('Model Size (MB)')
plt.legend(title='Model Category')
plt.show()


In [ ]:
# Function to create summary tables
def create_summary_table(df, category):
    summary = df.groupby(['pruning_method', 'pruning_percentage']).agg({
        'accuracy': 'mean',
        'model_size_MB': 'mean',
        'sparsity_percent': 'mean',
        'inference_time_sec': 'mean'
    }).reset_index()
    summary['category'] = category.capitalize()
    return summary

# Create summary tables
pruned_summary = create_summary_table(pruned_df, 'pruned')
finetuned_summary = create_summary_table(finetuned_df, 'finetuned')
kd_summary = create_summary_table(kd_df, 'kd')

# Combine summaries
all_summaries = pd.concat([pruned_summary, finetuned_summary, kd_summary], ignore_index=True)

# Display the summary table
print(all_summaries)


In [ ]:
# Create a pivot table for Accuracy
accuracy_pivot = all_summaries.pivot_table(
    index=['pruning_method', 'pruning_percentage'],
    columns='category',
    values='accuracy'
).reset_index()

print("Accuracy Comparison:")
print(accuracy_pivot)

# Similarly, create pivot tables for Inference Time
inference_time_pivot = all_summaries.pivot_table(
    index=['pruning_method', 'pruning_percentage'],
    columns='category',
    values='inference_time_sec'
).reset_index()

print("\nInference Time Comparison (sec/batch):")
print(inference_time_pivot)


In [ ]:
# Pivot for heatmap
accuracy_heatmap = accuracy_pivot.pivot("pruning_method", "pruning_percentage", "Finetuned")
plt.figure(figsize=(10, 8))
sns.heatmap(accuracy_heatmap, annot=True, fmt=".2f", cmap="YlGnBu")
plt.title('Finetuned Model Accuracy Heatmap')
plt.xlabel('Pruning Percentage')
plt.ylabel('Pruning Method')
plt.show()


In [ ]:
# Melt the inference_time_pivot for seaborn
inference_time_melted = inference_time_pivot.melt(id_vars=['pruning_method', 'pruning_percentage'],
                                                var_name='category',
                                                value_name='inference_time_sec')

plt.figure(figsize=(12, 8))
sns.barplot(data=inference_time_melted, x='pruning_percentage', y='inference_time_sec', hue='category')
plt.title('Inference Time Comparison Across Model Categories')
plt.xlabel('Pruning Percentage')
plt.ylabel('Avg Inference Time per Batch (sec)')
plt.legend(title='Model Category')
plt.show()


In [ ]:
g = sns.FacetGrid(combined_df, col="pruning_method", hue="category", col_wrap=3, height=4, aspect=1)
g.map(sns.lineplot, "pruning_percentage", "accuracy", marker="o")
g.add_legend()
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Accuracy Across Pruning Methods and Categories')
plt.show()


In [ ]:
def plot_grouped_bar(df, metric, title, ylabel):
    plt.figure(figsize=(14, 8))
    sns.barplot(data=df, x='pruning_percentage', y=metric, hue='category', ci=None)
    plt.title(title)
    plt.xlabel('Pruning Percentage')
    plt.ylabel(ylabel)
    plt.legend(title='Model Category')
    plt.show()

# Plotting Accuracy Comparison
plot_grouped_bar(combined_df, 'accuracy', 'Accuracy Comparison Across Categories', 'Accuracy (%)')

# Plotting Inference Time Comparison
plot_grouped_bar(combined_df, 'inference_time_sec', 'Inference Time Comparison Across Categories', 'Inference Time per Batch (sec)')

# Plotting Sparsity Comparison
plot_grouped_bar(combined_df, 'sparsity_percent', 'Sparsity Comparison Across Categories', 'Sparsity (%)')

# Plotting Model Size Comparison
plot_grouped_bar(combined_df, 'model_size_MB', 'Model Size Comparison Across Categories', 'Model Size (MB)')


In [ ]:
def create_summary_table(df, category):
    summary = df.groupby(['pruning_method', 'pruning_percentage']).agg({
        'accuracy': 'mean',
        'model_size_MB': 'mean',
        'sparsity_percent': 'mean',
        'inference_time_sec': 'mean'
    }).reset_index()
    summary['category'] = category.capitalize()
    return summary

# Create summary tables
pruned_summary = create_summary_table(pruned_df, 'pruned')
finetuned_summary = create_summary_table(finetuned_df, 'finetuned')
kd_summary = create_summary_table(kd_df, 'kd')

# Combine summaries
all_summaries = pd.concat([pruned_summary, finetuned_summary, kd_summary], ignore_index=True)

# Display the summary table
print(all_summaries)


In [ ]:
# Pivot for Accuracy
accuracy_pivot = all_summaries.pivot_table(
    index=['pruning_method', 'pruning_percentage'],
    columns='category',
    values='accuracy'
).reset_index()

print("Accuracy Comparison:")
print(accuracy_pivot)

# Pivot for Inference Time
inference_time_pivot = all_summaries.pivot_table(
    index=['pruning_method', 'pruning_percentage'],
    columns='category',
    values='inference_time_sec'
).reset_index()

print("\nInference Time Comparison (sec/batch):")
print(inference_time_pivot)
